In [1]:
import numpy as np
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 6
MAX_LIST_LEN = 6
MAX_STEPS = 120
SUCCESS_REWARD = 10.0
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.98
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma098_step120'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = -0.01
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = -0.02
            else:
                response_token = vocab['more']
                reward = -0.01
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = 0.1
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -0.1
            else:
                reward = -0.01
            self.indices = None
        else:
            reward = -1.0
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    # Optionally, load a checkpoint
    # load_checkpoint("path_to_checkpoint", model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        state = 'expect_action'
        done = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = -1.0
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = -1.0
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = -1.0
                done = True

            rewards.append(reward)
        #
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / (episode_cnt + 0.00001)
            episode_cnt = 0
            total_reward = 0.0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{avg_reward:.4f}.pth")
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns = []
            R = 0
            gamma = GAMMA
            for r in rewards[::-1]:
                R = r + gamma * R
                returns.insert(0, R)
            returns = torch.tensor(returns).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss: {loss.item():.4f}, total reward: {sum(rewards):.4f}, {t2-t1} sec")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss: -6.3606, total reward: -0.9200, 0.09358549118041992 sec
Episode 1, loss: -0.3700, total reward: -1.0000, 0.004981279373168945 sec
Episode 2, loss: -3.3361, total reward: -0.9100, 0.008436203002929688 sec
Episode 3, loss: -10.5306, total reward: -0.8400, 0.020661592483520508 sec
Episode 4, loss: -30.1035, total reward: -0.9800, 0.07024240493774414 sec
Episode 5, loss: -47.4489, total reward: -1.2500, 0.0953364372253418 sec
Episode 6, loss: -8.9686, total reward: -0.9400, 0.0214388370513916 sec
Episode 7, loss: -37.4582, total reward: -1.2200, 0.07979321479797363 sec
Episode 8, loss: -12.2345, total reward: -0.3900, 0.10324811935424805 sec
Episode 9, loss: 0.0541, total reward: -0.0900, 0.10496091842651367 sec
Episode 10, loss: -9.2524, total reward: -0.3900, 0.10399293899536133 sec
Episode 11, loss: 0.2073, total reward: -0.0900, 0.10507798194885254 sec
Episode 12, loss: -14.3039, total reward: -0.5800, 0.1052086353302002 sec
Episode 13, loss: -7.2800, total reward: -0.

Episode 111, loss: 2.8904, total reward: 0.2300, 0.10985112190246582 sec
Episode 112, loss: -1.9336, total reward: -1.0000, 0.008538007736206055 sec
Episode 113, loss: -6.0208, total reward: -0.1800, 0.09936189651489258 sec
Episode 114, loss: 114.4847, total reward: 10.0400, 0.027384042739868164 sec
Episode 115, loss: -5.1915, total reward: -0.9200, 0.012547731399536133 sec
Episode 116, loss: -2.1030, total reward: 0.0300, 0.0980064868927002 sec
Episode 117, loss: -6.3912, total reward: -0.2400, 0.10170364379882812 sec
Episode 118, loss: -30.8365, total reward: -0.8000, 0.0777590274810791 sec
Episode 119, loss: -2.1235, total reward: -1.0000, 0.00784444808959961 sec
Episode 120, loss: -3.6183, total reward: -0.1900, 0.0955955982208252 sec
Episode 121, loss: -0.9042, total reward: 0.2200, 0.1038045883178711 sec
Episode 122, loss: 102.5285, total reward: 10.0300, 0.0315401554107666 sec
Episode 123, loss: 2.7702, total reward: 0.1200, 0.09867453575134277 sec
Episode 124, loss: 289.6273, t

Episode 222, loss: 0.1402, total reward: 0.0100, 0.0991981029510498 sec
Episode 223, loss: -1.9693, total reward: -0.0900, 0.10426020622253418 sec
Episode 224, loss: -1.2167, total reward: 0.1200, 0.1056373119354248 sec
Episode 225, loss: 3.3047, total reward: 0.2200, 0.10426044464111328 sec
Episode 226, loss: 6.8127, total reward: 0.3200, 0.1103215217590332 sec
Episode 227, loss: -10.2798, total reward: -0.6600, 0.03155875205993652 sec
Episode 228, loss: -17.1201, total reward: -0.7000, 0.0405881404876709 sec
Episode 229, loss: -2.8495, total reward: 0.1100, 0.10246753692626953 sec
Episode 230, loss: -3.9173, total reward: 0.0100, 0.10557794570922852 sec
Episode 231, loss: -1.6699, total reward: 0.0100, 0.10772919654846191 sec
Episode 232, loss: -28.0476, total reward: -0.8600, 0.10210657119750977 sec
Episode 233, loss: -2.0835, total reward: 0.0100, 0.10726141929626465 sec
Episode 234, loss: 0.0429, total reward: 0.1200, 0.10942912101745605 sec
Episode 235, loss: 8.1881, total reward

Episode 333, loss: -3.3140, total reward: -0.0500, 0.10777044296264648 sec
Episode 334, loss: 5.1703, total reward: 0.3200, 0.11042022705078125 sec
Episode 335, loss: 2.0383, total reward: 0.2200, 0.10749244689941406 sec
Episode 336, loss: 274.0443, total reward: 10.2500, 0.09708118438720703 sec
Episode 337, loss: -2.5608, total reward: 0.1900, 0.1080939769744873 sec
Episode 338, loss: -3.0078, total reward: -0.1500, 0.10808801651000977 sec
Episode 339, loss: 4.3027, total reward: 0.3200, 0.10934591293334961 sec
Episode 340, loss: 2.3482, total reward: 0.2200, 0.10942435264587402 sec
Episode 341, loss: 3.0560, total reward: 0.1200, 0.10946202278137207 sec
Episode 342, loss: 323.6197, total reward: 10.2400, 0.09818053245544434 sec
Episode 343, loss: -1.3477, total reward: 0.1200, 0.10454845428466797 sec
Episode 344, loss: 0.9591, total reward: 0.2200, 0.10493898391723633 sec
Episode 345, loss: -0.9734, total reward: 0.1100, 0.10500693321228027 sec
Episode 346, loss: 245.2849, total rewa

...

Episode 8740, loss: 69.0080, total reward: 10.4500, 0.09009194374084473 sec
Episode 8741, loss: 128.4767, total reward: 10.4800, 0.08035659790039062 sec
Episode 8742, loss: 94.3516, total reward: 10.7700, 0.08570623397827148 sec
Episode 8743, loss: 112.0444, total reward: 10.1600, 0.07367992401123047 sec
Episode 8744, loss: 81.7338, total reward: 10.2500, 0.0864717960357666 sec
Episode 8745, loss: 0.4780, total reward: 0.1100, 0.09774374961853027 sec
Episode 8746, loss: 107.4774, total reward: 10.4500, 0.09657692909240723 sec
Episode 8747, loss: 0.2957, total reward: 0.2200, 0.09833049774169922 sec
Episode 8748, loss: 3.0117, total reward: 0.2900, 0.09902644157409668 sec
Episode 8749, loss: 0.4209, total reward: 0.2200, 0.09921765327453613 sec
Episode 8750, loss: 55.8483, total reward: 10.2800, 0.07932090759277344 sec
Episode 8751, loss: 126.6503, total reward: 10.4200, 0.06633520126342773 sec
Episode 8752, loss: 143.6343, total reward: 10.3800, 0.07879400253295898 sec
Episode 8753, lo

Episode 8849, loss: 117.7209, total reward: 10.1200, 0.09121346473693848 sec
Episode 8850, loss: -1.1764, total reward: 0.1100, 0.09776592254638672 sec
Episode 8851, loss: 113.0715, total reward: 10.0500, 0.08580517768859863 sec
Episode 8852, loss: 1.6000, total reward: 0.3200, 0.09795069694519043 sec
Episode 8853, loss: 113.0122, total reward: 10.5000, 0.07607007026672363 sec
Episode 8854, loss: 125.2215, total reward: 10.0900, 0.0725250244140625 sec
Episode 8855, loss: 170.9623, total reward: 10.1700, 0.07979536056518555 sec
Episode 8856, loss: 1.4335, total reward: 0.4200, 0.09717583656311035 sec
Episode 8857, loss: 1.3742, total reward: 0.3900, 0.0986185073852539 sec
Episode 8858, loss: 1.2765, total reward: 0.5200, 0.10017538070678711 sec
Episode 8859, loss: 128.4129, total reward: 10.2800, 0.07969999313354492 sec
Episode 8860, loss: 69.0237, total reward: 10.4000, 0.040972232818603516 sec
Episode 8861, loss: 144.9751, total reward: 10.3200, 0.09466004371643066 sec
Episode 8862, l

Episode 8958, loss: 0.2228, total reward: 0.0200, 0.09889769554138184 sec
Episode 8959, loss: -1.8671, total reward: 0.0100, 0.09845304489135742 sec
Episode 8960, loss: 1.0477, total reward: 0.3200, 0.0987389087677002 sec
Episode 8961, loss: 135.8581, total reward: 10.3500, 0.09175992012023926 sec
Episode 8962, loss: 57.7577, total reward: 10.2000, 0.040377140045166016 sec
Episode 8963, loss: 82.2419, total reward: 10.1500, 0.05127429962158203 sec
Episode 8964, loss: 4.3550, total reward: 0.3200, 0.09719276428222656 sec
Episode 8965, loss: 93.0634, total reward: 10.2700, 0.051026105880737305 sec
Episode 8966, loss: 40.2974, total reward: 10.2600, 0.01968240737915039 sec
Episode 8967, loss: 107.4739, total reward: 10.3800, 0.04344654083251953 sec
Episode 8968, loss: -1.1685, total reward: 0.0300, 0.09582400321960449 sec
Episode 8969, loss: 0.6634, total reward: 0.1100, 0.09802675247192383 sec
Episode 8970, loss: 1.1524, total reward: 0.3200, 0.0986795425415039 sec
Episode 8971, loss: 47

Episode 9065, loss: 88.6026, total reward: 10.4700, 0.08055329322814941 sec
Episode 9066, loss: 2.2245, total reward: 0.3200, 0.09680962562561035 sec
Episode 9067, loss: 100.2422, total reward: 10.4300, 0.09635639190673828 sec
Episode 9068, loss: -1.1826, total reward: -0.0900, 0.09761786460876465 sec
Episode 9069, loss: 111.1357, total reward: 10.3300, 0.09604859352111816 sec
Episode 9070, loss: 120.0863, total reward: 10.1000, 0.07016277313232422 sec
Episode 9071, loss: 2.5613, total reward: 0.5200, 0.09836435317993164 sec
Episode 9072, loss: 140.8225, total reward: 10.4000, 0.07354998588562012 sec
Episode 9073, loss: 35.0650, total reward: 10.0200, 0.030762434005737305 sec
Episode 9074, loss: 101.3530, total reward: 10.6300, 0.062166452407836914 sec
Episode 9075, loss: 0.1151, total reward: 0.3200, 0.09639883041381836 sec
Episode 9076, loss: 3.9788, total reward: 0.3200, 0.10004973411560059 sec
Episode 9077, loss: -0.1608, total reward: 0.2200, 0.09857511520385742 sec
Episode 9078, 

Episode 9175, loss: 123.1722, total reward: 10.3600, 0.053743839263916016 sec
Episode 9176, loss: 140.5811, total reward: 10.4500, 0.055100440979003906 sec
Episode 9177, loss: 80.8363, total reward: 10.4600, 0.05221986770629883 sec
Episode 9178, loss: 63.3753, total reward: 10.3900, 0.04150748252868652 sec
Episode 9179, loss: 109.8993, total reward: 10.7900, 0.07698702812194824 sec
Episode 9180, loss: 100.4136, total reward: 10.6800, 0.08107757568359375 sec
Episode 9181, loss: -1.4098, total reward: 0.0100, 0.09631919860839844 sec
Episode 9182, loss: 76.8641, total reward: 10.2200, 0.03410458564758301 sec
Episode 9183, loss: 143.3116, total reward: 10.3600, 0.04981732368469238 sec
Episode 9184, loss: 89.0651, total reward: 10.2500, 0.0846858024597168 sec
Episode 9185, loss: 76.6161, total reward: 9.8900, 0.0385584831237793 sec
Episode 9186, loss: 125.3135, total reward: 10.2400, 0.05499076843261719 sec
Episode 9187, loss: 1.6699, total reward: 0.5200, 0.09652829170227051 sec
Episode 91